# ET Example



In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import os
import time
import h5py
import matplotlib.pyplot as plt
import numpy as np
from sgvb_psd.psd_estimator import PSDEstimator

HERE = os.path.dirname(os.path.abspath(__file__))

def load_et_data() -> np.ndarray:
    """
    Return (n_samples, 3) array of XYZ channels
    """
    files_and_keys = [
        ("data/X_ETnoise_GP.hdf5", 'E1:STRAIN'),
        ("data/Y_ETnoise_GP.hdf5", 'E2:STRAIN'),
        ("data/Z_ETnoise_GP.hdf5", 'E3:STRAIN')
    ]
    channels = [h5py.File(os.path.join(HERE, file), "r")[key][:] for file, key in files_and_keys]
    return np.column_stack(channels)

def load_et_data():
    """
    Return (n_samples, 3) array of XYZ channels
    """
    channels = []
    with h5py.File(os.path.join(HERE, "data/et_data.h5"), "r") as f:
        for key in ["X", "Y", "Z"]:
            channels.append(f[key][:])
    return np.column_stack(channels)


data = load_et_data()

2024-09-19 12:44:03.702657: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


12:44:10|SGVB-PSD|INFO| 0s |No GPU detected. Running on CPU.


NameError: name '__file__' is not defined

In [2]:
N_theta = 400
start_time = time.time()
optim = PSDEstimator(
    N_theta=N_theta,
    nchunks=125,
    fs=2048,
    ntrain_map=100,
    x=data,
    max_hyperparm_eval=1,
    fmax_for_analysis=128,
    degree_fluctuate=N_theta,
    seed=0,
)

optim.run()
end_time = time.time()
estimation_time = end_time - start_time
print('The estimation time is', estimation_time, 'seconds')

NameError: name 'data' is not defined

In [ ]:
optim.plot_coherence(labels='XYZ')
plt.savefig("ET_coherence.png")

optim.plot_psd(labels='XYZ')
plt.savefig("ET_psd.png")

optim.plot_vi_losses()
plt.savefig("ET_vi_losses.png")